## Author(s)

- Author1 = {"name": "Julie Zhang", "affiliation": "University of British Columbia", "email": "juliezyw@gmail.com"}

# Building a Data Pipeline

In the first assignment we were posed with the question "is cashtag frequency tied to increases in stock price?".  To invistigate this we obtained some data from the New York Stock Exchange (NYSE), that consisted of the stock ticker symbols for various stocks trading on the NYSE.  We also used a dataset from Cresci *et al* (2019), that contained a large number of tweets with embedded stock tickers, represented as *cashtags*.

For this assignment we are going to work through the process of accessing the Twitter data from our DBMS, aligning `datetime` twitter fields with the datetime information about daily stock values.  This will allow us to perform several calculations and perform a visual assessment of our data.

We want to:

  1. Obtain information about the frequency of tweeting for *all* stock tickers, to understand if tweeting patterns are common across all stocks.
  2. Produce an interactive plot for a subset of stock tickers (using the [plotly package](https://plotly.com/python/) for Python).
  3. Summarise the net change in stock price as a function of tweet volume for various stocks.

1. Set up your analysis here.  Import the packages you'll be using set up the connection and a cursor.  The connection is *essentially* the same as last time, only this time you will be using a database called `loadeddata`, not `rawdata`.

In [4]:
# Put your code here.  More than likely you can copy/paste right in.
# Make sure you have your .env file in the same folder as this project file.

import os
import psycopg2
from dotenv import load_dotenv
import numpy
import sparklines
import pandas as pd
import plotly.express as px

load_dotenv()
conString = {'host':os.environ.get('DB_HOST'),
             'dbname':os.environ.get('DB_NAME'),
             'user':os.environ.get('DB_USER'),
             'password':os.environ.get('DB_PASS'),
             'port':os.environ.get('DB_PORT')}


1a.  The tweets are in the `loadeddata` database's `tweets.tweets` table.  We've previously connected to the server, but we're connecting to a new database within the server, so you need to adjust the `DB_NAME` field in you `.env` file to account for the change.  For each tweet we need to extract the actual tweet symbol.

A value in the `text` column of the `tweets.tweets` table looks like this:

`Keybank National Association OH Has $400,000 Stake in ChemoCentryx Inc $CCXI https://t.co/JZleFRCYKj`

Look at the Postgres documentation for [regular expressions](https://www.postgresql.org/docs/12/functions-matching.html) and [string functions](https://www.postgresql.org/docs/12/functions-string.html) (look at `LTRIM`, `SUBSET`, and `regexp_matches` in particular).  Write a query to extract only a stock ticker symbol from a tweet (excluding the dollar value). Use this query to extract the first 100 stock tickers into your active python session and print them as a list (e.g. `['GE','VNCE','RGEN']`).  Use a comment to explain how the query you've written works.

In [5]:
conn = psycopg2.connect(**conString)
cur = conn.cursor()
Ticker=[]
cur.execute("select LTRIM(unnest(regexp_matches(text, '\$[A-Z]+\M','g')),'$') as Ticker from tweets.tweets limit 100;")
for i in range(100):
    ##use ''.join to convert tuple to string, then added to list Ticker
    Ticker.append(''.join(cur.fetchone()))
Ticker

['SPY',
 'QQQ',
 'MET',
 'VDRM',
 'ELED',
 'QCOM',
 'ASNA',
 'ENDP',
 'FRZT',
 'CHRO',
 'ICNB',
 'AMRN',
 'BAC',
 'INSY',
 'JNJ',
 'XGTI',
 'INTC',
 'QQQ',
 'ZNGA',
 'BBRY',
 'YHOO',
 'CSCO',
 'WMT',
 'AMZN',
 'WFM',
 'MS',
 'MU',
 'WBA',
 'SHIP',
 'ECOB',
 'AFOM',
 'SPYR',
 'VDRM',
 'AMGN',
 'SRPT',
 'PTCT',
 'NVAX',
 'ICPT',
 'ADXS',
 'PBYI',
 'PPHM',
 'MRK',
 'INSY',
 'CLVS',
 'FRZT',
 'ICNB',
 'CHRO',
 'ELED',
 'MU',
 'DCTH',
 'ALDR',
 'MBRX',
 'MU',
 'MU',
 'VRX',
 'S',
 'NDRM',
 'ADP',
 'MU',
 'CTL',
 'MU',
 'ELGX',
 'FFIV',
 'BOX',
 'MSFT',
 'BABA',
 'KSS',
 'BABA',
 'ITGS',
 'SBFM',
 'MBRX',
 'TOPS',
 'TMPSW',
 'BBY',
 'ALSN',
 'CHRO',
 'PSID',
 'ITUS',
 'HLRTF',
 'RWLK',
 'PNOW',
 'KIWB',
 'GLDN',
 'CBIO',
 'AVEO',
 'CNAT',
 'HRS',
 'VRSK',
 'CHRO',
 'PSID',
 'ITUS',
 'HLRTF',
 'RWLK',
 'PNOW',
 'KIWB',
 'GLDN',
 'CBIO',
 'AVEO',
 'HRL',
 'NEM']

1b. Examine these first 100 stock tickers and the text that they were exracted from.  Do you notice issues?  Are all the results accurate?  **NOTE**: There are for sure some records that are wrong.  If, for whatever reason, you do not find any errors in regex detection/extraction in your first 100 rows, use `LIMIT 100 OFFSET 100` to look at the next set of 100 results.

ANSWER:the regulary expression we used in sql query does not guarantee 100% match of cashtags. for example if we have $$SPY, then the cashtag can not be extracted properly.

1c. Given the answer above, what are some considerations we must make in cleaning the data?  How can we minimize the issues we find?

ANSWER: during data cleaning, we need to watch for outliers, data gaps, misspellings, incorrectly entered, duplication and etc. To minimize the issues, we need to start at the earliest stage, make a clear cleaning plan and think from the customer's perspective.

We should check the extracted stock against the set of stock tickers, to make sure it's in the valid set (we can use referential integrity between the symbol table and the extracted cashtags).  We can also perform some estimates on error rates.

2. Now that we are able to extract stock data (to some degree), let's avoid doing major data cleaning just yet.  First, lets look at the stock data and find out, for each stock, how many tweets have been made about the stock, and how many days the stock has been tweeted about.  Here I'd like you to build a SQL query that returns the stock symbol (as above), the count of tweets, and the count of distinct dates on which the symbol has been tweeted.  But we're going to break it down here:

2a. You should notice that the `createdat` column in the table is a `text` field in Postgres that looks something like this: `Thu May 18 22:00:02 +0000 2017`. To convert this field you need to use the Postgres function [`TO_DATE`](https://www.postgresql.org/docs/12/functions-formatting.html) (you can also use `TO_TIMESTAMP`, but our stock data is aggregated by day, so we'll stick with day-level records here).  Write a SQL query to extract the date for only one row and execute the query.

In [41]:
sql="select to_date(createdat,'DY Mon DD HH24:MI:SS +0000 YYYY') from tweets.tweets limit 1;"
cur.execute(sql)
date=cur.fetchall()
format(date[0][0])

'2017-06-24'

2b. In the `tweets` schema you'll see a number of tables.  The process of cleaning the data was accomplished as in the Jupyter notebook `data_cleaning.ipynb` that you have been given already.  In the `tweets` schema there are a number of tables: `cleantweets` is the table of all (non-retweet) tweets, there is a table `symbols`, `stockvalues`, `cashtags` and `retweets`.  

Using these tables, create a query to return the count of each cashtag symbol instance on each day in the dataset.  Return the first 10 rows.

In [55]:
sql="""select t.createdate::date date, s.nasdaqsymbol symbol, count(c.symbolid) 
from tweets.cashtags c
join tweets.symbols s on c.symbolid=s.symbolid
join tweets.cleantweets t on c.tweetid=t.tweetid
group by t.createdate::date, s.nasdaqsymbol,c.symbolid;"""
df=pd.read_sql(sql,conn)

In [56]:
df.head(10)

,date,symbol,count
0,2017-05-24,A,44
1,2017-05-24,AA,5
2,2017-05-24,AAMC,7
3,2017-05-24,AAN,1
4,2017-05-24,AAP,69
5,2017-05-24,AAT,30
6,2017-05-24,AB,5
7,2017-05-24,ABB,9
8,2017-05-24,ABBV,47
9,2017-05-24,ABC,13


3. A [sparkline](https://en.wikipedia.org/wiki/Sparkline) is a graphical representation of data proposed by Edward Tufte.  We're going to start out with sparklines because they're fairly easy to implement in Python, and we don't need to use anything particularly crazy to put them in.  We're going to make sparklines for a number of records, but to do that, first we need the set of distinct symbols, the set of distinct dates, and then a table with symbols, days, and the count of tweets for that combination.

a. You now have a list of all valid stock symbols, with the count of tweets per day.  With these three objects we can itterate through each stock. We want to use the [`sparklines`](https://pypi.org/project/sparklines/) package to print out something that looks like this:

In [ ]:
# import sparklines -- You can have this here, or up above where you initialize everything.

' '.join([''.join(sparklines.sparklines([0,10,2,3,10,2,6,6,4,12])), 'Stock name'])

Given the format that `sparklines` expects, we should take the input data, that looks something like this (your order may vary):

```
[('PRGE', 1986, datetime.date(2017, 5, 18)), ('PRGE', 1004, datetime.date(2017, 5, 19)), ... ]
```

And get it to look something like this (each entry in the second list is one of the days over which our stock data stretches):

```python
result = ['PRGE', [0, 0, 0, 1986, 0, 1004, 0, 0, 0, 0, 0, 0, 97, 492, 245, 0, 833, 5188, 12587, 10312, 1173, 4267, 2081, 22, 42, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 266, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 552, 508, 88, 541, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 693, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
```

This way we can print out a sparkline like this:

```
print(' '.join([''.join(sparklines.sparklines(result[1])), result[0]]))
```

**So, in the next block, you will generate a sparkline for the first ten stock tickers in the dataset (either use a `for` loop with a counter so you can `break` from the loop when the counter gets to 10, use a `range` statement, or use something else you know about).**

You've already copied all the stock tweets into memory in the block above, so you can use that, and you can get a list of the ordered days (we know they go from May to September, 2017), possibly using a [`set()`](https://docs.python.org/3/tutorial/datastructures.html#sets) of all the days in the record.  So, for ten different stock symbols, create a list with the `[stock name, [counts of tweets per day]]` for each date in the dataset (this may be more days than the number of days on which the stock symbol was tweeted).

Likely, you will need to use the [`.index()` method](https://docs.python.org/3/library/array.html#array.array.index).  There are several solutions to this problem, but more than likely you will be using `map` and `filter` functions here (or similar `pandas` functions).  So go for it.  Return 10 sparklines:

In [57]:
#use pandas dataframes to constuct the list required for the first 10 tickers
for i in range(10):
    s=df.iloc[i]['symbol']
    result=[s]
    result.append(df[df['symbol']==s]['count'].tolist())
    print(' '.join([''.join(sparklines.sparklines(result[1])), result[0]]))

▂▄▃▂▂▆▇▇▇▆▃▃▃▃▄▅▅▅▂▄▃▁▂▃▂▂▃▂▂▅▆▃▄▄▃▃▃▃▃▃▃▆▅▄▄▃▃▂▃▂▃▄▂▃▃▃▃▄▂▂▃▂▂▃▂▁▃▅█▄▃▄▄▃▃▃▃▄▄▃▅▆▅▄▃▂ A
▁▂▂▁▁▃▃▄▃▄▂▂▄▂▃▃▅▂▁▃▂▁▂▁▁▂▂▁▃▅▃▄▃▄▂▃▃▃▂▃▄▅▅▄▃▂▃▃▁▃▄▃▃▄▄██▂▂▂▂▂▂▁▄▃▃▃▂▃▂▃▄▄▃▅▂▂▄▄█▃▄▂ AA
▃▃▁▃█▃▃▂▁▁▂▂▂▁▁▁▂▁▁▂▁▃▂▄▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▂▁▁▁▂▂▁▂▁▁▂▁▂▂▄▁▁▃ AAMC
▁▂▂▃▃▃▅▄▇▄▅▃▂▄▇▇▅▁▆▂▁▂▂▁▄▂▁▆▆▅▆▅▃▅▄▃▄▁▂▄▇▄█▇▂▄▆▂▂▂▂▃▄▃▆▄▂▁▂▄▂▁▃▂▄▄▃▃▁▃▃▂▃▄▁▂▇▆▄▅▆▁ AAN
▂▂▂▁▂▃▄▅▃▂▁▁▂▂▂▃▂▂▁▂▂▁▁▁▁▁▁▁▂▂▁▁▁▂▂▂▂▁▁▁▂█▅▆▂▁▂▂▁▁▁▁▁▂▁▁▂▂▁▁▁▁▁▁▁▂█▅▃▃▂▂▃▂▁▁▂▁▁▂▁▂▁▁▁ AAP
▄▃▂▁▁▃▅▇▃▂▁▁▂▃▃▃▃▂▄▂▂▃▂▂▂▂▃▂▁▂▃▁▂▁▂▁▁▁▁▂▅▃▁▃▂▁▂▄▂▁▂▂▂▃▂▁▁▁▃▂▁▁▂▂▂▂▆▂▄▂▂▂▄▃▅▃▃▄█▆▃▃ AAT
▁▃▂▁▁▃▆█▃▄▂▂▅▃▆▃█▄▁▄▄▁▂▁▁▂▁▁▄▄▂▂▂▃▂▂▃▁▁▃▂▂▁▅▁▁▁▂▂▃▂▃▂▃▂▃▅▂▂▁▃▂▂▂▁▃▂▃▆▄▂▁▃▁▃▅█▄▄▄█▅▃▂ AB
▁▂▁▁▁▂▄▇▄▄▂▂▁▂▃▅▅▂▁▄▄▁▂▁▁▂▂▁▆▃▂▂▂▂▂▂▂▄▂▃▄▂▄▃▁▁▂▂▁▁▂▁▁▂▁▃█▃▁▁▂▃▂▁▂▂▃▂▂▂▂▂▂▃▃▄▂▃▂▄▃▁▂ ABB
▂▂▁▁▁▄▅▇▄▅▄▄▄▃▅▆█▅▃▄▃▂▃▂▂▃▃▂▇▇▃▃▄▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▂▂▂▄▂▂▁▂▂▂▁▁▂▂▂▂▃▂▂▂▂▂▂▄▃▂▃▄▃▃▃▁ ABBV
▁▂▁▁▁▅▇▇▄▅▂▅▃▂▅▅▆▅▁▃▄▃▂▃▂▃▂▁█▃▂▃▂▂▂▂▄▂▂▁▃▅▆▄▂▂▃▁▂▂▁▁▂▂▁▂▂▂▁▁▁▂▂▁▃▄▃▃▅▅▂▂▂▃▂▂▃▃▃▃▄▂▃▂ ABC


3c. Given the sparklines you've generated, describe any of the patterns you see.  Do you see potential issues with the data?  You can run the code a few times to get a different set of sparklines, but please describe two general patterns you see (i.e., not specific to a single stock).  For each of the two patterns, explain how you might address these.

ANSWER:
Pattern 1 is discussion rised but quickly cooled off for a long period, the flucuation is very high; Pattern 2 (AAN) shows consistent high level of discussions throughout the whole time range.

4. Interactive plots can improve the legibility of data representations, and provide additional context for data, however the principles of good data management must still be followed.  We can use `plotly` to create an interactive plot for several tickers.  Using `pandas` makes some of the data management a bit easier as well.  For example:

```
import plotly.express as px
query = """SELECT sy.nasdaqsymbol,
                  date_trunc('day', ct.createdate),
                  COUNT(sy.nasdaqsymbol) AS total
           FROM          tweets.cashtags AS ctg
           INNER JOIN     tweets.symbols AS sy ON ctg.symbolid = sy.symbolid
           INNER JOIN tweets.cleantweets AS ct ON ct.tweetid = ctg.tweetid
           GROUP BY sy.nasdaqsymbol, date_trunc('day', ct.createdate)
           ORDER BY date_trunc('day', ct.createdate)"""

cur = conn.cursor()
cur.execute(query)
stocktweets = cur.fetchall()

# We know the order of the tuples in the stocktweets list, because
# it matches the SQL query we sent.
stockdf = pd.DataFrame(stocktweets, columns=['Symbol', 'Date', 'Count'])

# Now limit it to four stock tweets:
stocksmall = stockdf[stockdf['Symbol'].isin(['A','B','C','D'])]

fig = px.line(stocksmall, 
              x='Date',
              y='Count', 
              color = 'Symbol', 
              title='Stock Tweet Volume by Day', 
              log_y = True)
fig.show()
```

Based on the code (above), create a line plot that shows the 10 stocks with the highest total number of tweets (you will likely have to do a subquery for this).

In [58]:
conn = psycopg2.connect(**conString)
sql="""select distinct (rf.symbol), rf.count, rf.date from (
select  distinct on (s.nasdaqsymbol) s.nasdaqsymbol as symbol, t.createdate::date date, count(c.symbolid) 
from tweets.cashtags c
join tweets.symbols s on c.symbolid=s.symbolid
join tweets.cleantweets t on c.tweetid=t.tweetid
group by t.createdate::date, s.nasdaqsymbol,c.symbolid) rf
order by rf.count desc
limit 10;"""
df2=pd.read_sql(sql,conn)
df2

,symbol,count,date
0,TWTR,1806,2017-06-22
1,ETH,1557,2017-07-19
2,LTC,868,2017-07-06
3,IBM,750,2017-06-09
4,BABA,700,2017-08-30
5,BCH,600,2017-08-27
6,SPY,584,2017-08-14
7,XOM,502,2017-06-22
8,PSTG,484,2017-08-25
9,LLY,465,2017-05-31


In [59]:
stocklist=df2['symbol'].to_list()
stocksmall=df[df['symbol'].isin(stocklist)]
fig = px.line(stocksmall, 
              x='date',
              y='count', 
              color = 'symbol', 
              title='Stock Tweet Volume by Day', 
              log_y = True)
fig.show()

4b. In the code above we use the [`date_trunc()`](https://www.postgresql.org/docs/12/functions-datetime.html) function in Postgres.  Explain why we use it.  Look at the graph you've made.  Are there any patterns you see in the data?  Discuss them and potential issues they may cause in our analysis.

ANSWER: 
date_trunc() will only parse information down to the instructed level. In ths case, date_trunc('day', ct.createdate) will return 
